### Imports

In [8]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [9]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [10]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [11]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [12]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [13]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-55    8.1   38     0.94   0        Ruim     1.00      
-57    3.8   4      0.39   1        Bom      0.99      
-86    5.9   83     0.40   0        Ruim     1.00      
-62    7.8   66     0.39   0        Ruim     1.00      
-90    7.0   96     0.93   0        Ruim     1.00      
-36    2.5   36     0.23   1        Bom      0.99      
-56    6.4   11     0.50   0        Ruim     0.72      
-66    5.7   32     0.63   0        Ruim     0.99      
-36    4.0   5      0.07   1        Bom      1.00      
-69    9.2   43     0.99   0        Ruim     1.00      
-65    4.1   82     0.89   0        Ruim     1.00      
-44    1.2   31     0.33   1        Bom      0.99      
-58    9.6   35     0.77   0        Ruim     1.00      
-70    7.2   33     0.82   0        Ruim     1.00      
-57    9.4   62     0.89   0        Ruim     1.00      
-65    6.3   45     0.52   0        Ruim     0.9

### Save

In [14]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
